In [39]:
# C:\Users\15005\Downloads\get_set5_psnr-master\Set5
import os
import math
import random
import numpy as np
import torch
import cv2
from torchvision.utils import make_grid
from datetime import datetime
# import torchvision.transforms as transforms



IMG_EXTENSIONS = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']


def get_image_paths(dataroot):
    paths = None  # return None if dataroot is None
    if dataroot is not None:
        paths = sorted(_get_paths_from_images(dataroot))
    return paths


def _get_paths_from_images(path):
    assert os.path.isdir(path), '{:s} is not a valid directory'.format(path)
    images = []
    for dirpath, _, fnames in sorted(os.walk(path)):
        for fname in sorted(fnames):
            if is_image_file(fname):
                img_path = os.path.join(dirpath, fname)
                images.append(img_path)
    assert images, '{:s} has no valid image file'.format(path)
    return images


# ----------------------------------------
# get single image of size HxWxn_channles (BGR)
# ----------------------------------------
def read_img(path):
    # read image by cv2
    # return: Numpy float32, HWC, BGR, [0,1]
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)  # cv2.IMREAD_GRAYSCALE
    img = img.astype(np.float32) / 255.
    if img.ndim == 2:
        img = np.expand_dims(img, axis=2)
    # some images have 4 channels
    if img.shape[2] > 3:
        img = img[:, :, :3]
    return img


# ----------------------------------------
# get uint8 image of size HxWxn_channles (RGB)
# ----------------------------------------
def imread_uint(path, n_channels=3):
    #  input: path
    # output: HxWx3(RGB or GGG), or HxWx1 (G)
    if n_channels == 1:
        img = cv2.imread(path, 0)  # cv2.IMREAD_GRAYSCALE
        img = np.expand_dims(img, axis=2)  # HxWx1
    elif n_channels == 3:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)  # BGR or G
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # GGG
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # RGB
    return img

def imread_uint2(path, n_channels=3):
    #  input: path
    # output: HxWx3(RGB or GGG), or HxWx1 (G)
    if n_channels == 1:
        img = cv2.imread(path, 0)  # cv2.IMREAD_GRAYSCALE

        img = np.expand_dims(img, axis=2)  # HxWx1
    elif n_channels == 3:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)  # BGR or G
        img = cv2.resize(img, (512, 512))
        print(img.shape)
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # GGG
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # RGB
    return img

def imsave(img, img_path):
    if img.ndim == 3:
        img = img[:, :, [2, 1, 0]]
    cv2.imwrite(img_path, img)
    
    
# ----------
# PSNR
# ----------
def calculate_psnr(img1, img2, border=0):
    # img1 and img2 have range [0, 255]
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    h, w = img1.shape[:2]
    img1 = img1[border:h-border, border:w-border]
    img2 = img2[border:h-border, border:w-border]

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


# ----------
# SSIM
# ----------
def calculate_ssim(img1, img2, border=0):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    h, w = img1.shape[:2]
    img1 = img1[border:h-border, border:w-border]
    img2 = img2[border:h-border, border:w-border]

    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')


def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)
    
import os
import cv2

HR_path = './Untitled Folder'
SR_path = './result'
#HR_path = 'dataset/benchmark/Urban100/HR'
# SR_path = 'experiments/results/Urban100/x4'
#SR_path = 'experiments/results/csnla_Urban100'


n_channels = 3

def evulate_diff_name():
    hr_paths = get_image_paths(HR_path)
    numbers = len(hr_paths)
    sum_psnr = 0
    max_psnr = 0
    min_psnr = 100
    sum_ssim = 0
    max_ssim = 0
    min_ssim = 1
    for hr_path in hr_paths:
        name, ext = os.path.splitext(os.path.basename(hr_path))
        img_name = os.path.basename(hr_path)
        print(img_name)
        temp = str(name) + '_x4_SR.png'
        print(temp)
        sr_path = os.path.join(SR_path, temp)
        print(sr_path)
        print(hr_path)
        print(sr_path)
        img_Hr = imread_uint(hr_path, n_channels=n_channels)  # HR image, int8
        img_Sr = imread_uint2(sr_path, n_channels=n_channels)  # HR image, int8
        print(img_Hr.shape)
        print(img_Sr.shape)
        #         img_Hr = cv2.imread(hr_path)
#         img_Sr = cv2.imread(sr_path)
        psnr = calculate_psnr(img_Sr, img_Hr,)
        print(psnr)
        sum_psnr += psnr
        max_psnr = max(max_psnr,psnr)
        min_psnr = min(min_psnr, psnr)
        ssim = calculate_ssim(img_Sr, img_Hr,)
        # print(ssim)
        sum_ssim += ssim
        max_ssim = max(max_ssim,ssim)
        min_ssim = min(min_ssim, ssim)
    print('Average psnr = ', sum_psnr / numbers)
    print('min_psnr = ', min_psnr)
    print('Max_psnr = ', max_psnr)
    print('Average ssim = ', sum_ssim / numbers)
    print('min_ssim = ', min_ssim)
    print('Max_ssim = ', max_ssim)


if __name__ == '__main__':
    print('-------------------------compute psnr and ssim for evulate sr model---------------------------------')
    # evulate()
    evulate_diff_name()




-------------------------compute psnr and ssim for evulate sr model---------------------------------
baby.png
baby_x4_SR.png
./result\baby_x4_SR.png
./Untitled Folder\baby.png
./result\baby_x4_SR.png
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
35.576958280739895
Average psnr =  35.576958280739895
min_psnr =  35.576958280739895
Max_psnr =  35.576958280739895
Average ssim =  0.9484449234094189
min_ssim =  0.9484449234094189
Max_ssim =  0.9484449234094189
